In [3]:
import requests
from bs4 import BeautifulSoup
import csv

In [13]:
# URL of the webpage to scrape
url = input("Enter CR Link: ")

# Requesting the webpage content
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

# Extract state and parliamentary constituency name
header = soup.find("div", class_="page-title")
if header:
    h2_tag = header.find("h2")
    if h2_tag:
        # Extract text from the span and strong tags within the h2 tag
        span_tag = h2_tag.find("span")
        strong_tag = h2_tag.find("strong")
        
        if span_tag and strong_tag:
            # Get the raw text of pc_name and state
            pc_name_raw = span_tag.get_text(strip=True).strip()
            state = strong_tag.get_text(strip=True).strip('()')
            
            # Remove leading numeric characters, hyphens, and state information from pc_name
            pc_name = pc_name_raw.split('-')[1].split('(')[0].strip()

        else:
            raise ValueError("Could not find the span or strong tag with state and pc_name information")
    else:
        raise ValueError("Could not find the h2 tag with state and pc_name information")
else:
    raise ValueError("Could not find the header with state and pc_name")

# Extract the table data
table = soup.find("table")  # Adjust the selector if necessary
if not table:
    raise ValueError("Could not find the results table")

rows = table.find_all("tr")[1:]  # Exclude header row

data = []
total_votes_sum = 0  # Initialize the sum of total votes

def parse_int(value):
    return int(value.replace(",", "")) if value != '-' else 0

def parse_float(value):
    return float(value) if value != '-' else 0.0

for row in rows:
    cols = row.find_all("td")
    if len(cols) < 7:  # Ensuring there are enough columns
        continue

    evm_votes = parse_int(cols[3].get_text(strip=True))
    postal_votes = parse_int(cols[4].get_text(strip=True))
    total_votes = parse_int(cols[5].get_text(strip=True))
    per_votes = parse_float(cols[6].get_text(strip=True))
    
    total_votes_sum += total_votes  # Add to the sum of total votes
    
    data.append([
        state,
        pc_name,
        cols[1].get_text(strip=True),  # Candidate
        cols[2].get_text(strip=True),  # Party
        evm_votes,
        postal_votes,
        total_votes,
        per_votes
    ])

# Add the sum of total votes as a new column for each row
for row in data:
    row.append(total_votes_sum)

# Save the data into a CSV file with a combined name
file_name = f"CR_{state}_{pc_name.replace(' ', '_')}.csv"
with open(file_name, "w", newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["state", "pc_name", "candidate", "party", "evm_votes", "postal_votes", "total_votes", "per_votes", "sum_totalvotes"])
    writer.writerows(data)

print(f"Data extracted and saved to {file_name}")


KeyboardInterrupt: Interrupted by user